In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import pandas as pd
from src.inference import fetch_next_hour_predictions, load_batch_of_features_from_store
from src.plot_utils import plot_aggregated_time_series

current_date = pd.Timestamp.now(tz="Etc/UTC")

features = load_batch_of_features_from_store(current_date)

2025-03-04 10:24:22,926 INFO: Initializing external client
2025-03-04 10:24:22,926 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 10:24:23,684 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214690
Fetching data from 2025-02-03 15:24:22.924136+00:00 to 2025-03-04 14:24:22.924136+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.73s) 


In [4]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-04 14:00:00
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2025-03-04 14:00:00
2,1,7,5,3,2,1,1,5,1,0,...,5,16,11,17,6,1,2,2,4,2025-03-04 14:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-04 14:00:00
4,2,4,1,0,4,0,0,1,1,2,...,5,7,14,3,3,4,2,1,7,2025-03-04 14:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,1,0,0,0,0,0,0,0,0,0,...,1,2,0,0,0,0,0,0,259,2025-03-04 14:00:00
247,1,1,0,0,2,0,0,0,0,0,...,1,2,5,1,1,2,1,1,260,2025-03-04 14:00:00
248,37,28,33,48,54,39,24,38,18,9,...,1,9,12,15,31,27,28,27,261,2025-03-04 14:00:00
249,79,97,95,105,104,53,45,21,18,6,...,87,155,104,140,132,103,96,89,262,2025-03-04 14:00:00


In [5]:
predictions = fetch_next_hour_predictions()

2025-03-04 10:24:33,534 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 10:24:33,592 INFO: Initializing external client
2025-03-04 10:24:33,592 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 10:24:34,163 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214690
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.36s) 
Current UTC time: 2025-03-04 15:24:33.532942+00:00
Next hour: 2025-03-04 16:00:00+00:00
Found 251 records


In [6]:
predictions


,pickup_location_id,predicted_demand,pickup_hour
0,252,0.0,2025-03-04 16:00:00+00:00
1,243,0.0,2025-03-04 16:00:00+00:00
2,224,2.0,2025-03-04 16:00:00+00:00
3,117,0.0,2025-03-04 16:00:00+00:00
4,69,0.0,2025-03-04 16:00:00+00:00
...,...,...,...
246,47,0.0,2025-03-04 16:00:00+00:00
247,82,0.0,2025-03-04 16:00:00+00:00
248,240,0.0,2025-03-04 16:00:00+00:00
249,245,0.0,2025-03-04 16:00:00+00:00


In [7]:
top10 = (
    predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].to_list()
)

In [8]:
top10

[233, 113, 231, 246, 143, 90, 144, 79, 137, 151]

In [9]:
top10

[233, 113, 231, 246, 143, 90, 144, 79, 137, 151]

In [10]:
features["pickup_location_id"] == 22

0      False
1      False
2      False
3      False
4      False
       ...  
246    False
247    False
248    False
249    False
250    False
Name: pickup_location_id, Length: 251, dtype: bool

In [11]:
features_cp = features.copy()

In [12]:
from datetime import timedelta
features = features_cp[features_cp["pickup_location_id"] == 161]
prediction = predictions[predictions["pickup_location_id"] == 161]
time_series_columns = [
    col for col in features.columns if col.startswith("rides_t-")
]
time_series_values = [features[col].iloc[0] for col in time_series_columns] + prediction["predicted_demand"].to_list()

# Convert pickup_hour Series to single timestamp
pickup_hour = pd.Timestamp(features["pickup_hour"].iloc[0])

# Generate corresponding timestamps for the time series
time_series_dates = pd.date_range(
    start=pickup_hour - timedelta(hours=len(time_series_columns)),
    end=pickup_hour,
    freq="h",
)

# Create a DataFrame for the historical data
historical_df = pd.DataFrame(
    {"datetime": time_series_dates, "rides": time_series_values}
)

In [13]:
historical_df

,datetime,rides
0,2025-02-04 14:00:00,398.0
1,2025-02-04 15:00:00,404.0
2,2025-02-04 16:00:00,422.0
3,2025-02-04 17:00:00,546.0
4,2025-02-04 18:00:00,592.0
...,...,...
668,2025-03-04 10:00:00,255.0
669,2025-03-04 11:00:00,297.0
670,2025-03-04 12:00:00,312.0
671,2025-03-04 13:00:00,381.0
